In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from sklearn.datasets.samples_generator import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Classificação com NN

Com aquilo que aprendemos, vamos tentar resolver um problema de classificação binária. Veja como é simples apenas modificando um pouco a estrutura de nossa rede.

In [0]:
from sklearn.datasets import make_classification

In [0]:
X, y = make_classification(n_samples = 100000)

In [0]:
X[0]

In [0]:
y[0]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .5)

In [0]:
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)

In [0]:
X_train[0]

Vamos criar nossa função `build_model` verifique que agora temos um problema de classificação binário, sendo assim, iremos utilizar como função de loss a `binary_crossentropy`.

In [0]:
def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(20, input_dim = 20, activation=tf.nn.relu))
    model.add(keras.layers.Dense(40, activation=tf.nn.relu))
    model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

Verifique que modificamos também nossa função de otimização e métrica. Para entender melhor como o algoritmo de otimização "adam" funciona, veja o seguinte [artigo](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/).

Vamos modificar nossa função para plotar os gráficos

In [0]:
def plot_history(history):
  plt.figure(figsize = (15, 10))
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.plot(history.epoch, np.array(history.history['acc']), label='Train Acc')
  plt.plot(history.epoch, np.array(history.history['val_acc']), label = 'Val Acc')
  plt.legend()

In [0]:
model = build_model()

history = model.fit(X_train, y_train, epochs = 5, validation_split = 0.2, verbose = False)

plot_history(history)

In [0]:
[loss, acc] = model.evaluate(X_test, y_test)

print("Acc: {:.4f}".format(acc))

In [0]:
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_curve, auc

Podemos plotar nossa matriz de confusão:

In [0]:
test_preds = model.predict(X_test).flatten()
test_preds_bin = (test_preds > 0.5)
test_preds_bin

In [0]:
cm  = confusion_matrix(y_test, test_preds_bin)

sns.heatmap(cm, annot = True, fmt='g', cmap = 'winter')

## Curva ROC

A Curva de operação do receptor (*Receiver Operationg Characteristic*)  é uma representação gráfica que ilustra o desempenho (ou performance) de um sistema classificador binário e como o seu limiar de discriminação é variado. [Wikipedia](https://pt.wikipedia.org/wiki/Caracter%C3%ADstica_de_Opera%C3%A7%C3%A3o_do_Receptor)

 ![PIC6](https://ncss-wpengine.netdna-ssl.com/wp-content/uploads/2016/06/ROC-Curves-Empirical-19.png)

#### Sensitivity

Também chamada de *true positive rate*, *recall* ou *probabilidade de detecção*. Mede a proporção de positivos realmente detectados como positivos.

$$
TPR = \frac{TP}{TP + FN}
$$

#### Specificity

Também chamada de *true negative rate*, mede a proporção de negativos realmente detectados como negativos.

$$
TNR = \frac{TN}{TN + FP}
$$

Obtendo os valores:

In [0]:
fpr, tpr, thresholds = roc_curve(y_test, test_preds)
auc_calc = auc(fpr, tpr)

In [0]:
plt.figure(figsize = (15, 10))
plt.plot(fpr, tpr, label='Modelo (area = {:.3f})'.format(auc_calc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Curva ROC')
plt.legend(loc='best')
plt.plot([0, 1], [0, 1], 'k--')

## Desafio

Utilize o dataset fornecido a tente criar um modelo capaz de classificar os dados não conhecidos pela coluna diagnóstico.

In [0]:
df = pd.read_csv('https://github.com/pgiaeinstein/aula-09/raw/master/df_map.csv')

In [0]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [0]:
notebook_seed = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = notebook_seed)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
df.head()

In [0]:
def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(20, input_dim = 20, activation=tf.nn.relu))
    model.add(keras.layers.Dense(40, activation=tf.nn.relu))
    model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

# Trabalhando com arquivos DICOM

## O que é DICOM?

- DICOM - Digital Imaging and Communications in Medicine;

- Conjunto de normas para tratamento, armazenamento e transmissão de informação médica (PROTOCOLO);
 - Padroniza a formatação das imagens diagnósticas como tomografias, ressonâncias magnéticas, radiografias, ultrassonografias, etc...;
  - Permite que imagens médicas e informações associadas sejam trocadas entre equipamentos de diagnóstico geradores de imagens, computadores e hospitais, ou seja, estabelece uma linguagem comum entre os equipamentos de diferentes marcas.

[Wikipedia](https://pt.wikipedia.org/wiki/DICOM)

## Protocolo?

- Convenção que controla e possibilita uma conexão, comunicação, transferência de dados entre dois sistemas computacionais.

- Define regras de sintaxe, semântica e sincronização da comunicação.

---

| Protocolo | Descrição |
|--|--|
| IP | [Internet Protocol](https://pt.wikipedia.org/wiki/Internet_Protocol "Internet Protocol") |
| HTTP | [Hypertext Transfer Protocol](https://pt.wikipedia.org/wiki/HTTP "HTTP") |
| POP3 | [Post Office Protocol](https://pt.wikipedia.org/wiki/Post_Office_Protocol "Post Office Protocol") |
| SMTP | [Simple Mail Transfer Protocol](https://pt.wikipedia.org/wiki/Simple_Mail_Transfer_Protocol "Simple Mail Transfer Protocol") |


## PyDICOM

In [0]:
!wget "https://github.com/pgiaeinstein/aula-09/raw/master/img1.dcm"
!wget "https://github.com/pgiaeinstein/aula-09/raw/master/img2.dcm"

In [0]:
# É necessário instalar o pacote pydicom pelo gerenciador de pacotes pip
!pip install pydicom

In [0]:
# Após isso, podemos importar o pacote
import pydicom
import pylab

In [0]:
dcmdata = pydicom.read_file('./img1.dcm')
print(dcmdata)

In [0]:
dcmimg = dcmdata.pixel_array
print(dcmimg.dtype)
print(dcmimg.shape)

In [0]:
plt.figure(figsize=(20,10))
plt.imshow(dcmimg, cmap=pylab.cm.binary)
plt.axis('off')

## Desafio

Repita o processo acima para o arquivo img2.dcm

# Sobel

In [0]:
! wget "https://res.cloudinary.com/demo/image/upload/e_grayscale/happy_dog.jpg"
! wget "https://images.freeimages.com/images/large-previews/308/grayscale-rose-1478187.jpg"
! wget "https://openclipart.org/image/2400px/svg_to_png/225455/Carl-Georg-Adolph-Hasenpflug-Medieval-Town-Grayscale.png"
! wget "http://floodplanuk.org/wp-content/uploads/2018/08/Images.jpg"

In [0]:
import os
import functools

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['axes.grid'] = False
mpl.rcParams['figure.figsize'] = (12,12)
mpl.style.use('fivethirtyeight')

from sklearn.model_selection import train_test_split
import matplotlib.image as mplimg
from PIL import Image

import tensorflow as tf
import tensorflow.contrib as tfcontrib

from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import backend as K

import cv2

In [0]:
sobel_img_1 = './happy_dog.jpg'
sobel_img_2 = './grayscale-rose-1478187.jpg'
sobel_img_3 = './Carl-Georg-Adolph-Hasenpflug-Medieval-Town-Grayscale.png'
sobel_img_4 = './Images.jpg'

In [0]:
def aplica_sobel(imagem):
  
    img = cv2.imread(imagem, 0)
  
    sobel_x = cv2.Sobel(img, cv2.CV_64F, 1, 0)
    sobel_y = cv2.Sobel(img, cv2.CV_64F, 0, 1)
    sobel_g = np.sqrt(sobel_x**2 + sobel_y**2)
    sobel_g *= 255.0 / np.max(sobel_g)

    plt.figure(1, figsize = (15, 15))
    plt.subplot(221),plt.imshow(img, cmap = 'gray')
    plt.title('Input Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(222),plt.imshow(sobel_x, cmap = 'gray')
    plt.title('Sobel X'), plt.xticks([]), plt.yticks([])
    plt.subplot(223),plt.imshow(sobel_y, cmap = 'gray')
    plt.title('Sobel Y'), plt.xticks([]), plt.yticks([])
    plt.subplot(224),plt.imshow(sobel_g, cmap = 'gray')
    plt.title('Sobel G'), plt.xticks([]), plt.yticks([])
    plt.show()

In [0]:
aplica_sobel(sobel_img_1)

## Desafio

Tente aplicar o filtro sobel nas imagens `sobel_img_2`. `sobel_img_3` e `sobel_img_4`.

# MNIST

O objetivo desta atividade é familiarizar o aluno com uma CNN. Iremos utilizar o dataset __MNIST__ para treinar uma CNN para classificar números manuscritos.

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, MaxPooling2D
from keras.layers.convolutional import Conv2D 
from keras.utils import np_utils
from keras import backend as K

In [0]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
nb_classes = 10

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

Vamos exibir algumas imagens deste dataset

In [0]:
plt.figure(figsize=(10, 10))

for i in range(10):

    plt.subplot(1, 10, i + 1)
    plt.axis('off')
    plt.imshow(X_train[i])
#     plt.imshow(X_train[i, :, :, 0])

plt.show()

In [0]:
img_rows, img_cols = 28, 28

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [0]:
nb_filters = 32
kernel_size = (3, 3)
pool_size = (2, 2)

model = Sequential()

model.add(Conv2D(nb_filters, kernel_size, padding='valid', input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(nb_filters, kernel_size))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(units=128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(units=nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

In [0]:
epochs = 5

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=128, verbose=1)

In [0]:
def mostrar_erros(predictions, trueclass=None, predictedclass=None, maxtoshow=10):
   
    rounded = np.argmax(predictions, axis=1)
    errors = rounded!=y_test

    ii = 0
    plt.figure(figsize=(maxtoshow, 1))
    for i in range(X_test.shape[0]):
        if ii>=maxtoshow:
            break
        if errors[i]:
            if trueclass is not None and y_test[i] != trueclass:
                continue
            if predictedclass is not None and predictions[i] != predictedclass:
                continue
            plt.subplot(1, maxtoshow, ii+1)
            plt.axis('off')
            plt.imshow(X_test[i,:,:,0], cmap="gray")
            plt.title("%d (%d)" % (rounded[i], y_test[i]))
            ii = ii + 1

In [0]:
preds = model.predict(X_test)

mostrar_erros(preds)

In [0]:
mostrar_erros(preds, trueclass = 3)

# Kmeans

Em nossa próxima atividade, vamos tentar segmentar uma imagem utilizando uma U-Net.

Para possibilitar o treinamento desta rede, precisamos criar máscaras de segmentação, ou seja, o que esperamos que a rede responda quando receber uma imagem para analisar.

Uma forma simples de fazer isso é aplicar K-means.

In [0]:
!wget "https://github.com/pgiaeinstein/aula-10/raw/master/original_imgs.zip"
!unzip original_imgs.zip
!pip install tqdm

In [0]:
from sklearn.cluster import KMeans

def label_para_pixel(bloco, labels, l, a):
    dimensao = bloco.shape[1]
    imagem = np.zeros((l, a, dimensao))
    index = 0
        
    if bloco[0][0] > bloco[1][0]:
        cor_cell = [1., 1., 1.]
        cor_fundo = [0., 0., 0.]
    else:
        cor_cell = [0., 0., 0.]
        cor_fundo = [1., 1., 1.]
    
    for i in range(l):
        for j in range(a):
            if labels[index] == 0:
                imagem[i][j] = cor_fundo
            else:
                imagem[i][j] = cor_cell
            index += 1
    return imagem

def retorna_kmeans(n_cores, img_path):
    
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img = np.array(img, dtype = np.float64) / 255

    w, h, d = tuple(img.shape)
    assert d == 3
    image_array = np.reshape(img, (w * h, d))

    kmeans = KMeans(n_clusters = n_cores, random_state = 0).fit(image_array)
    labels = kmeans.predict(image_array)
    img_mask = label_para_pixel(kmeans.cluster_centers_, labels, w, h)
    return img, img_rgb, img_mask
    
    
def plota_imagens(img, img_mask):
    plt.figure(1, figsize = (15, 15))
    ax1 = plt.subplot(121)
    ax1.axis('off')
    ax1.set_title('Imagem original')
    ax1.imshow(img)

    ax2 = plt.subplot(122)
    ax2.axis('off')
    ax2.set_title('Imagem K-Means')
    ax2.imshow(img_mask)

Vamos imprimir o resultado da clusterização de uma imagem:

In [0]:
img = './original_imgs/BloodImage_00002.jpg'

img, img_rgb, img_mask = retorna_kmeans(2, img)
plota_imagens(img_rgb, img_mask)

Vamos automatizar a criação de nossa base de treino e teste.

A função abaixo percorrerá todo o conteúdo da pasta de imagens original e aplicará o processo de clusterização separando em imagem original e máscara criada em duas pastas distintas para facilitar o _import_ dos dados no exercício futuro.

In [0]:
import os
import errno
from tqdm import tqdm

IMAGENS_PATH = './imgs'
MASKS_PATH = './masks'

WORK_PATH = './original_imgs'

try:
    os.makedirs(IMAGENS_PATH)
    os.makedirs(MASKS_PATH)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
        
lista_imagens = os.listdir(WORK_PATH)

for imagem in tqdm(lista_imagens):
    img_nome = imagem.replace('.jpg', '')
    img, _, img_mask = retorna_kmeans(2, os.path.join(WORK_PATH, imagem))
    cv2.imwrite(os.path.join(IMAGENS_PATH, '{0}.jpg'.format(img_nome)), img * 255)
    cv2.imwrite(os.path.join(MASKS_PATH, '{0}_mask.jpg'.format(img_nome)), img_mask * 255)

# CNN

Vamos implementar uma U-Net e nosso objetivo é conseguir segmentar as imagens de células sanguíneas que processamos no exemplo anterior.

In [0]:
img_dir = './imgs'
label_dir = './masks'

X_train_imgs = os.listdir(img_dir)
y_train_imgs = os.listdir(label_dir)

for index, img_path in enumerate(X_train_imgs):
    X_train_imgs[index] = os.path.join(img_dir, img_path)
    
for index, img_path in enumerate(y_train_imgs):
    y_train_imgs[index] = os.path.join(label_dir, img_path)  

Vamos separar nossa base em treino e teste:

In [0]:
seed = 42
tam_test = 0.2

X_train_imgs, X_test_imgs, y_train_imgs, y_test_imgs = train_test_split(X_train_imgs, 
                                                                        y_train_imgs, 
                                                                        test_size = tam_test, 
                                                                        random_state = seed)

In [0]:
qtd_train_imgs = len(X_train_imgs)
qtd_test_imgs = len(X_test_imgs)

print("Quantidade de imagens para treino: {0}".format(qtd_train_imgs))
print("Quantidade de imagens para validação: {0}".format(qtd_test_imgs))
print("Quantidade total de imagens: {0}".format(qtd_train_imgs + qtd_test_imgs))

Podemos imprimir algumas imagens para verificar se não há nada de errado até aqui:

In [0]:
qtd_imgs_plot = 5

random_imgs = np.random.choice(qtd_train_imgs, qtd_imgs_plot)

plt.figure(figsize=(10, 20))

for i in range(0, qtd_imgs_plot * 2, 2):
    
    img_index = random_imgs[i // 2]
    X_path = X_train_imgs[img_index]
    y_path = y_train_imgs[img_index]

    plt.subplot(qtd_imgs_plot, 2, i + 1)
    plt.axis('off')
    plt.imshow(mplimg.imread(X_path))
    plt.title("Imagem original")

    img_mask = Image.open(y_path)
    # mask_mtx = np.unique(img_mask)

    plt.subplot(qtd_imgs_plot, 2, i + 2)
    plt.axis('off')
    plt.imshow(img_mask)
    plt.title("Máscara da imagem")  

plt.show()

Vamos criar um _pipeline_ (uma série de procedimentos que deverá ser respeitado pelo modelo) para treino.

Aqui vamos implementar funções que nos permitem aplicar _image augmentation_ em nossa base.

In [0]:
img_shape = (256, 256, 3)
batch_size = 3
epochs = 5

def proc_img(img_path, mask_path):
    
    _img = tf.read_file(img_path)
    img = tf.image.decode_jpeg(_img, channels = 3)

    _mask_img = tf.read_file(mask_path)
    mask_img = tf.image.decode_gif(_mask_img)[0]
    mask_img = mask_img[:, :, 0]
    mask_img = tf.expand_dims(mask_img, axis = -1)
    
    return img, mask_img

def shift_img(output_img, mask_img, width_shift_range, height_shift_range):
    if width_shift_range or height_shift_range:
        if width_shift_range:
            width_shift_range = tf.random_uniform([], -width_shift_range * img_shape[1],
                                                  width_shift_range * img_shape[1])
        if height_shift_range:
            height_shift_range = tf.random_uniform([], -height_shift_range * img_shape[0],
                                                   height_shift_range * img_shape[0])
        output_img = tfcontrib.image.translate(output_img, [width_shift_range, height_shift_range])
        mask_img = tfcontrib.image.translate(mask_img, [width_shift_range, height_shift_range])
    
    return output_img, mask_img

def flip_img(horizontal_flip, tr_img, mask_img):
    if horizontal_flip:
        flip_prob = tf.random_uniform([], 0.0, 1.0)
        tr_img, mask_img = tf.cond(tf.less(flip_prob, 0.5),
                                lambda: (tf.image.flip_left_right(tr_img), tf.image.flip_left_right(mask_img)),
                                lambda: (tr_img, mask_img))
    return tr_img, mask_img

def augment_img(img, mask_img, resize = None, scale = 1, hue_delta = 0, horizontal_flip = False,
             width_shift_range = 0, height_shift_range = 0):
    
    if resize is not None:
        mask_img = tf.image.resize_images(mask_img, resize)
        img = tf.image.resize_images(img, resize)

    if hue_delta:
        img = tf.image.random_hue(img, hue_delta)

    img, mask_img = flip_img(horizontal_flip, img, mask_img)
    img, mask_img = shift_img(img, mask_img, width_shift_range, height_shift_range)
    mask_img = tf.to_float(mask_img) * scale
    img = tf.to_float(img) * scale
    
    return img, mask_img

In [0]:
def get_baseline_dataset(filenames, labels, preproc_fn = functools.partial(augment_img), threads = 5, 
                         batch_size = batch_size, shuffle = True):           
    num_x = len(filenames)
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.map(proc_img, num_parallel_calls=threads)
    if preproc_fn.keywords is not None and 'resize' not in preproc_fn.keywords:
        assert batch_size == 1, "Imagens no batch não respeitam as mesmas dimensões"

    dataset = dataset.map(preproc_fn, num_parallel_calls=threads)

    if shuffle:
        dataset = dataset.shuffle(num_x)


    dataset = dataset.repeat().batch(batch_size)
    return dataset


In [0]:
train_cfg = {
    'resize': [img_shape[0], img_shape[1]],
    'scale': 1 / 255.,
    'hue_delta': 0.1,
    'horizontal_flip': True,
    'width_shift_range': 0.1,
    'height_shift_range': 0.1
}

train_preproc_function = functools.partial(augment_img, **train_cfg)

In [0]:
test_cfg = {
    'resize': [img_shape[0], img_shape[1]],
    'scale': 1 / 255.,
}

test_preproc_function = functools.partial(augment_img, **test_cfg)

In [0]:
train_dataset = get_baseline_dataset(X_train_imgs, y_train_imgs,
                                preproc_fn = train_preproc_function, batch_size = batch_size)

test_dataset = get_baseline_dataset(X_test_imgs, y_test_imgs, 
                              preproc_fn = test_preproc_function, batch_size = batch_size)

Podemos imprimir uma imagem de exemplo para verificar se nosso processo de _image augmentation_ está funcionando de maneira adequada:

In [0]:
tmp_dataset = get_baseline_dataset(X_train_imgs, y_train_imgs, preproc_fn = train_preproc_function,
                               batch_size = 1, shuffle = False)

img_aug_iterator = tmp_dataset.make_one_shot_iterator()
img_aug_prox = img_aug_iterator.get_next()

with tf.Session() as sess: 
    imgs_batch, _y = sess.run(img_aug_prox)

    plt.figure(figsize=(10, 10))
    img = imgs_batch[0]

    plt.subplot(1, 2, 1)
    plt.axis('off')
    plt.imshow(img)

    plt.subplot(1, 2, 2)
    plt.axis('off')
    plt.imshow(_y[0, :, :, 0])
    plt.show()

Vamos definir funções auxiliares para facilitar a construção do modelo:

In [0]:
def conv_block(tensor_entrada, qtd_filtros):
    encoder = layers.Conv2D(qtd_filtros, (3, 3), padding = 'same')(tensor_entrada)
    encoder = layers.BatchNormalization()(encoder)
    encoder = layers.Activation('relu')(encoder)
    encoder = layers.Conv2D(qtd_filtros, (3, 3), padding = 'same')(encoder)
    encoder = layers.BatchNormalization()(encoder)
    encoder = layers.Activation('relu')(encoder)
    return encoder

def encoder_block(tensor_entrada, qtd_filtros):
    encoder = conv_block(tensor_entrada, qtd_filtros)
    encoder_pool = layers.MaxPooling2D((2, 2), strides = (2, 2))(encoder)

    return encoder_pool, encoder

def decoder_block(tensor_entrada, tensor_merge, qtd_filtros):
    decoder = layers.Conv2DTranspose(qtd_filtros, (2, 2), strides = (2, 2), padding = 'same')(tensor_entrada)
    decoder = layers.concatenate([tensor_merge, decoder], axis = -1)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Activation('relu')(decoder)
    decoder = layers.Conv2D(qtd_filtros, (3, 3), padding = 'same')(decoder)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Activation('relu')(decoder)
    decoder = layers.Conv2D(qtd_filtros, (3, 3), padding = 'same')(decoder)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Activation('relu')(decoder)
    return decoder

In [0]:
inputs = layers.Input(shape = img_shape)
encoder0_pool, encoder0 = encoder_block(inputs, 32)
encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64)
encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128)
encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256)
encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512)
center = conv_block(encoder4_pool, 1024)
decoder4 = decoder_block(center, encoder4, 512)
decoder3 = decoder_block(decoder4, encoder3, 256)
decoder2 = decoder_block(decoder3, encoder2, 128)
decoder1 = decoder_block(decoder2, encoder1, 64)
decoder0 = decoder_block(decoder1, encoder0, 32)
outputs = layers.Conv2D(1, (1, 1), activation = 'sigmoid')(decoder0)

In [0]:
model = models.Model(inputs = [inputs], outputs = [outputs])

Vamos definir nossas funções de _loss_:

In [0]:
def dice_coeff(y_true, y_pred):
    smooth = 1.0
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
    loss = losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

In [0]:
model.compile(optimizer = 'adam', loss = bce_dice_loss, metrics = [dice_loss])
model.summary()

In [0]:
model_path = './modelo_bc_V1.hdf5'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = model_path, 
                                        monitor = 'val_dice_loss', 
                                        save_best_only = True, 
                                        verbose = True)

In [0]:
history = model.fit(train_dataset, 
                    steps_per_epoch = int(np.ceil(qtd_train_imgs / float(batch_size))),
                    epochs = epochs,
                    validation_data = test_dataset,
                    validation_steps = int(np.ceil(qtd_test_imgs / float(batch_size))),
                    callbacks = [model_checkpoint])

In [0]:
dice = history.history['dice_loss']
val_dice_loss = history.history['val_dice_loss']

loss = history.history['loss']
val_loss = history.history['val_loss']

qtd_epochs = range(epochs)

In [0]:
plt.figure(figsize = (16, 8))
plt.subplot(1, 2, 1)
plt.plot(qtd_epochs, dice, label = 'Dice Loss - Treinamento')
plt.plot(qtd_epochs, val_dice_loss, label = 'Dice Loss - Validação')
plt.legend(loc='upper right')
plt.title('Dice Loss - Treino e Validação')

plt.subplot(1, 2, 2)
plt.plot(qtd_epochs, loss, label = 'Loss - Treinamento')
plt.plot(qtd_epochs, val_loss, label = 'Loss - Validação')
plt.legend(loc='upper right')
plt.title('Loss - Treino e Validação')

plt.show()

In [0]:
model = models.load_model(model_path, custom_objects={'bce_dice_loss': bce_dice_loss,
                                                      'dice_loss': dice_loss})

In [0]:
img_aug_iterator = test_dataset.make_one_shot_iterator()
img_aug_prox = img_aug_iterator.get_next()

In [0]:
plt.figure(figsize=(20, 20))
for i in range(6):
    batch_of_imgs, label = tf.keras.backend.get_session().run(img_aug_prox)
    img = batch_of_imgs[0]
    predicted_label = model.predict(batch_of_imgs)[0]

    plt.subplot(6, 3, 3 * i + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title("Imagem de entrada")

    plt.subplot(6, 3, 3 * i + 2)
    plt.imshow(label[0, :, :, 0])
    plt.axis('off')
    plt.title("Máscara real")
    plt.subplot(6, 3, 3 * i + 3)
    plt.imshow(predicted_label[:, :, 0])
    plt.axis('off')
    plt.title("Output do modelo")
plt.show()